### Install libraries

In [1]:
import PyPDF2
from PyPDF2 import PdfReader
import boto3
import json

### Read in a PDF file

In [2]:
def read_pdf(pdf_file): # read pdf file and return text
    with open(pdf_file, 'rb') as file: # open pdf file
        reader = PyPDF2.PdfReader(file) # create reader object
        text = '' # create empty string to store text
        for page in reader.pages: # iterate over pages
            text += page.extract_text().replace('\n', ' ') # extract text and add to string. replace \n with space
        return text # return text

In [3]:
pdf_file_path = read_pdf('data/Corporate_Travel_Policy.pdf')

In [4]:
pdf_file_path

"Corporate Travel and Time Off Policy Introduction This policy establishes clear guidelines and procedures for time off and corporate travel for employees. It aims to ensure fair and consistent application throughout the organization while supporting operational needs. Annual Paid Time Off (PTO) Entitlement ● PTO Allocation: All employees receive five weeks (25 working days) of PTO per calendar year . ● Accrual of PTO: PTO accrues monthly based on the annual entitlement. ● Carryover: Unused PTO cannot be carried over to the next year . Employees are encouraged to utilize their PTO within the accrual year . Time Off Beyond PTO ● Managerial Approval: Additional time off beyond the allocated five weeks requires prior approval from the employee's direct manager . ● Request Procedure: Submit time off requests at least four weeks in advance for any period exceeding annual PTO. ● Considerations for Approval: Managers will assess the operational impact, employee performance and attendance, and

### Establish a connection to Bedrock Runtime

In [5]:
bedrock_runtime = boto3.client(region_name = 'us-east-1', 
                               service_name = 'bedrock-runtime')

### Engineer the prompt

In [6]:
pdf_text = read_pdf('data/Corporate_Travel_Policy.pdf')
user_question = "How many working days of PTO do employees get per calendar year?"

In [8]:
prompt = f"Answer the following question:{user_question}. Use this reference text for the answer:\n{pdf_text}"

### Call the prompt with a Bedrock model

In [9]:
titan_body = json.dumps({
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0,
        "maxTokenCount": 1024
    }
})

In [10]:
titan_response = bedrock_runtime.invoke_model(body=titan_body, modelId="amazon.titan-text-express-v1")
titan_response_body = json.loads(titan_response.get('body').read())
print(titan_response_body["results"][0]["outputText"])

 ● Health and Safety: Prioritize the health and safety of employees during travel by following company guidelines and local regulations. ● Return to Work: Upon return from travel, complete any necessary reporting and follow up on any outstanding tasks. By adhering to these policies, we can ensure that time off and corporate travel are managed effectively, promoting work-life balance, productivity, and a positive work environment.


### Call the prompt with the Llama2 model

In [11]:
llama_runtime = boto3.client(service_name='bedrock-runtime',
                             region_name='us-east-1')

In [19]:
llama_body = json.dumps({'prompt': prompt, 'max_gen_len': 1024, 'temperature': 0.9, 'top_p': 0.9})

In [20]:
llama_response = llama_runtime.invoke_model(body=llama_body, modelId='meta.llama2-13b-chat-v1')

In [21]:
# use the response body to get the output text
llama_response_body = json.loads(llama_response.get('body').read())

In [22]:
print(llama_response_body['generation'])


How many working days of PTO do employees get per calendar year?

According to the reference text, employees receive five weeks (25 working days) of PTO per calendar year.


### Functionalize the Llama2 approach

In [27]:
def get_pdf_answer(pdf_filepath, user_question):

    '''
    This function takes in a PDF file and a user question 
    and returns the answer to the question 
    using the text from the PDF file as reference.

    Parameters:
    pdf_file: variable type
    user_question: variable type

    Returns:
    str

    '''

    # Read in the PDF file
    reader = PdfReader(pdf_filepath)
    page = reader.pages[0]
    pdf_text = page.extract_text().replace('\n',' ')
    
    # Create the Bedrock Runtime client
    bedrock_runtime = boto3.client(region_name = 'us-east-1', 
                                   service_name = 'bedrock-runtime')
    
    # Create the prompt
    prompt = f"Answer the following question: {user_question}. Here is the reference text:\n{pdf_text}"

    # Create the body of the request
    body = json.dumps({'prompt': prompt, 
                                    'max_gen_len': 1024, 
                                    'temperature': 0.5, 
                                    'top_p': 0.9})
    
    # Make the request to the Bedrock Runtime client
    response = bedrock_runtime.invoke_model(body=body, modelId="meta.llama2-13b-chat-v1")

    # Use the response body to get the output text
    response_body = json.loads(response.get('body').read())

    # Return the output text
    return print(response_body['generation'])

In [28]:
# Call the function

get_pdf_answer('data/Corporate_Travel_Policy.pdf', 'How many working days of PTO do employees get per calendar year?')




How many working days of PTO do employees get per calendar year?

Based on the reference text, employees receive five weeks (25 working days) of PTO per calendar year.
